In [1]:
!pip install influxdb_client influxdb

In [2]:
#First add kafka consumer to consume dataset

In [3]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName("Test").getOrCreate()
df = pd.read_excel('SWaT_dataset_Jul 19 v2.xlsx', parse_dates = ['GMT +0']) #remove later
sdf = spark.createDataFrame(df)
sdf.show()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+--------------------+-----------+----------+------+--------+-----------+-----------+----------+--------+----------+----------+--------+--------+--------+--------+-----+--------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------+----------+----------+----------+----------+----------+------+------+------+------+--------+-----------+-----------+-------+----------+-----------+----------+--------+--------+-----------+-----------+-----------+-----------+-----+----------+----------+----------+----------+-----------+-----------+-----------+----------+------+------+------+------+--------+-----------+-----------+----------+----------+----------+----------+-------+-------+--------+--------+--------+-------+--------+-----------+-----------+-----------+
|              GMT +0|    FIT 101|   LIT 101|MV 101|P1_STATE|P101 Status|P102 Status|   AIT 201| AIT 202|   AIT 203|   FIT 201|  LS 201|  LS 202| LSL 203|LSLL 203|MV201|P2_STATE|P201 Status|P202

## Reservoir Sampling

In [8]:
import numpy as np
import random
    
class ReservoirSampling:
    def __init__(self, size):
        self.arr = np.zeros(size)
        self.count = 0
    def update(self,elem):
        # if counter less than the pool size we add this element in the first free position 
        if self.count < len(self.arr):
            for i in range(len(self.arr)):
                if self.arr[i]==0:
                    self.arr[i] = elem
                    self.count += 1
        # it counter >= pool size then we have to apply update algorithm
        r = random.randint(0, self.count-1)
        if r < len(self.arr):
            self.arr[r]=elem
        self.count += 1
    
    def ret(self):
        return self.arr

In [9]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

attack1 = np.array(sdf.select("FIT 401").collect())
sampling1 = ReservoirSampling(20)

for elem in attack1:
    sampling1.update(elem)
samples=sampling1.ret()

token = "i4mHlTHGTjH0nK974IYaZJztxhOdhwPhmcdYvZTVom05ZyrsJqwNLT3_6EUWPwi97nCG0oNx2QjMTlRapn5Q1g=="
org = "changeme"
bucket = "freq"

with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    dataCollect = sdf.collect()
    for elem in samples:
        p = influxdb_client.Point("samplingFIT401").field(str(elem), elem)
        write_api.write(bucket, org, p)    
            
            


## Misra Gries Algorithm
Finding frequent items determinastically. Given a bag b of n elements and an integer k ≥ 2. Find the values that occur more than n ÷ k times in b

In [10]:
from collections import Counter
def misra(arr, k):
    dic = {}
    for elem in arr:
        ## case 1: item already has counter or there are empty counters
        if elem in dic:
            dic[round(elem, 7)] +=1
        elif len(dic)<k:
            dic[round(elem, 7) ] = 1
        ## case 2: item doesn't have counter and there are no empty counters
        else:
            for key in list(dic.keys()):
                dic[round(key, 7)] -= 1
                if dic[round(key, 7)] == 0:
                    del dic[round(key, 7)]
    return dic



In [11]:
attack1 = np.array(sdf.select("FIT 401").collect())
flat_list = [item for sublist in attack1 for item in sublist]

In [12]:
data = misra(flat_list,15)

In [ ]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

token = "i4mHlTHGTjH0nK974IYaZJztxhOdhwPhmcdYvZTVom05ZyrsJqwNLT3_6EUWPwi97nCG0oNx2QjMTlRapn5Q1g=="
org = "changeme"
bucket = "freq"

with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    dataCollect = sdf.collect()
    for row in dataCollect:
        for key in data:
            p = influxdb_client.Point("freqFIT401").field(key, data[key])
            write_api.write(bucket, org, p)    
            
            


# BJKST Algorithm
Count distinct elements

In [ ]:
import math
class BJKST:
    def __init__(self, n, eps, c):
        self.h = UniversalHash(n)
        self.g = UniversalHash(int(math.log(n, 2)**2 * eps**(-4)))
        self.z = 0
        self.b = []
        self.eps = eps 
        self.c = c
    def update(self,elem):
        # if counter less than the pool size we add this element in the first free position 
        nzeros = np.count_nonzero(self.h.hash(elem)==0)
        if nzeros >= self.z:
            self.b.append([self.g.hash(elem) , nzeros])
        while len(self.b) >= self.c/self.eps**2:
            self.z += 1
            for i in self.b:
                if i[1] < self.z:
                    self.b.remove(i)
                    
    def ret(self):
        return len(self.b)*2**self.z

class UniversalHash:

    def __init__(self, space):
        self.random = random.Random()
        self.random.seed()
        self.space = space
    
    def hash(self,key):
        key = -(key + 1) if key < 0 else key
        return hash(key.__str__()) % self.space
    

In [ ]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

token = "i4mHlTHGTjH0nK974IYaZJztxhOdhwPhmcdYvZTVom05ZyrsJqwNLT3_6EUWPwi97nCG0oNx2QjMTlRapn5Q1g=="
org = "changeme"
bucket = "freq"

bjkst1 = BJKST(20,1,3)
attack1 = np.array(sdf.select("FIT 401").collect())
flat_list = [item for sublist in attack1 for item in sublist]

with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    dataCollect = sdf.collect()
    for i in flat_list:
        bjkst1.update(i)
        val = bjkst1.ret()
        p = influxdb_client.Point("difcountFIT401").field(str(val), val)
        write_api.write(bucket, org, p)    

In [ ]:
"""
def wingraphminer(stream, batch, windowsize, support):
    g = []
    q = []
    window = []
    R= []
    for i in range(0,len(stream),batch):
        data = stream[i:batch]
        c = coreset(data,support)
        window.append(c)
        if len(window) >= windowsize:
            R = window[0]
        else:
            R = []

def coreset(batch,support):
    ADS
"""             